# Final Project

W261 Machine Learning at Scale<br>
12 December 2018

Wei Wang;
Alice Lam;
John Tabbone;
Noah Randolph

### Question Formulation

Alice Lam: "enhancing CTR means improved monetization of your current traffic (eyeballs/views). The algorithm to predict CTR accurately is useful for the platform to show specific ads to specific people who would have the highest CTR"

### Algorithm Explanation

### EDA & Discussion of Challenges

In [55]:
%%writefile loadAndEDA.py
#!/usr/bin/env python

from pyspark.sql import types
from pyspark.sql.functions import udf

MAINCLOUDPATH = 'gs://w261_final_project/train.txt'
TOYCLOUDPATH = 'gs://w261_final_project/train_005.txt'
TOYLOCALPATH = 'data/train_005.txt'
SEED = 2615


# start Spark Session
from pyspark.sql import SparkSession
app_name = "row_counts"
spark = SparkSession\
        .builder\
        .appName(app_name)\
        .getOrCreate()
sc = spark.sparkContext


def loadData():
    # load the data into Spark dataframes
    # select path to data: MAINCLOUDPATH; TOYCLOUDPATH; TOYLOCALPATH
    df = spark.read.csv(path=TOYLOCALPATH, sep='\t')
    # change column names
    oldColNames = df.columns
    newColNames = ['Label']+['I{}'.format(i) for i in range(0,13)]+['C{}'.format(i) for i in range(0,26)]
    for oldName, newName in zip(oldColNames, newColNames):
        df = df.withColumnRenamed(oldName, newName)
    # change int column types to int from string
    for col in df.columns[:14]:
        df = df.withColumn(col, df[col].cast('int'))
    return df


def splitIntoTestAndTrain(df):
    '''randomly splits 80/20 into training and testing dataframes'''
    splits = df.randomSplit([0.2, 0.8], seed=SEED)
    testDf = splits[0]
    trainDf = splits[1]
    return testDf, trainDf


def displayHead(df, n=5):
    '''prints out head of the training dataset'''
    return df.head(n)


df = loadData()
testDf, trainDf = splitIntoTestAndTrain(df)
print("TEST DATASET ROW COUNTS: ", testDf.count())
print("TRAIN DATASET ROW COUNTS: ", trainDf.count())
print("HEAD\n", displayHead(trainDf))
print(df.dtypes)

Writing loadAndEDA.py


In [56]:
!python row_counts.py

2018-11-30 01:58:40 WARN  NativeCodeLoader:62 - Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
2018-11-30 01:58:42 WARN  Utils:66 - Service 'SparkUI' could not bind on port 4040. Attempting port 4041.
2018-11-30 01:58:52 WARN  Utils:66 - Truncated the string representation of a plan since it was too large. This behavior can be adjusted by setting 'spark.debug.maxToStringFields' in SparkEnv.conf.
TEST DATASET ROW COUNTS:  4578
TRAIN DATASET ROW COUNTS:  18379
HEAD
 [Row(Label=0, I0=None, I1=-2, I2=1, I3=0, I4=14, I5=None, I6=0, I7=16, I8=17, I9=None, I10=0, I11=None, I12=0, C0='3c9d8785', C1='207b2d81', C2='111fd16e', C3='7eac96d7', C4='30903e74', C5='fe6b92e5', C6='76fc09f3', C7='0b153874', C8='a73ee510', C9='3b08e48b', C10='0b60ef54', C11='fa8e4000', C12='0721132d', C13='b28479f6', C14='05619ef0', C15='

In [13]:
!python submit_job_to_cluster.py --project_id=w261-222623 --zone=us-central1-b --cluster_name=testcluster --gcs_bucket=w261_final_project --key_file=$HOME/w261.json --create_new_cluster --pyspark_file=row_counts.py --instance_type=n1-standard-4 --worker_nodes=2

Traceback (most recent call last):
  File "/anaconda3/lib/python3.5/site-packages/googleapiclient/discovery_cache/__init__.py", line 36, in autodetect
    from google.appengine.api import memcache
ImportError: No module named 'google.appengine'

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/anaconda3/lib/python3.5/site-packages/googleapiclient/discovery_cache/file_cache.py", line 33, in <module>
    from oauth2client.contrib.locked_file import LockedFile
ImportError: No module named 'oauth2client.contrib.locked_file'

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/anaconda3/lib/python3.5/site-packages/googleapiclient/discovery_cache/file_cache.py", line 37, in <module>
    from oauth2client.locked_file import LockedFile
ImportError: No module named 'oauth2client.locked_file'

During handling of the above exception, another exception occurred:

Traceback (

#### Results from running EDA code above:
Main dataset:<br>
('TEST DATASET ROW COUNTS: ', 9164811)<br>
('TRAIN DATASET ROW COUNTS: ', 36675806)<br>

Toy dataset:<br>
('TEST DATASET ROW COUNTS: ', 4578)<br>
('TRAIN DATASET ROW COUNTS: ', 18379)<br>

### Algorithm Implementation

### Application of Course Concepts